In [ ]:
'''
2. Neural Network Classifier with Keras
Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using Keras. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.
'''

In [2]:
# Training a multiclass Classifier
# Using Keras to construct a feedforward neural network with an output layer with soft-max activation functions

# Load libraries
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers

#import json library to read data in jsonl file
import json
#import pandas library
import pandas as pd

In [3]:
#read in the data as a dataframe
filename = "/home/arindam/Documents/mygithub/bu_dsc/data/raw/categorized-comments.jsonl"
with open(filename, 'r') as f:
    jsonl_list = list(f)

list1 = []
for obj in jsonl_list:
    res = json.loads(obj)
    list1.append(res)
    
comments = pd.DataFrame(list1)

#display the first few rows of data
comments.head()
# len(list1)

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


In [4]:
#Convert text to lowercase and romove punctuation
#define a function to clean the text
# import the required libraries here
#import regular expressions library
import re

def clean_text(text):
    """
    Remove punctuations and special characters, makes lower case
    Args: text
    Output: text
    """
    text=text.lower() #makes text lowercase
    text=re.sub('\\d|\\W+|_',' ',text) #removes extra white space
    text=re.sub('[^a-zA-Z]'," ", text) #removes any non-alphabetic characters
    return text

In [160]:
# Cleaning texts in the comments
# Using the transformed column for the model
# Testing the functions

sample_size = 10000
sample_cmnts = comments[:sample_size]
sample_cmnts['cleaned']=sample_cmnts['txt'].apply(clean_text)
# creating a dictionary to replace the string values to numeric
d = {'sports':1,'science_and_technology':2,'video_games':3}
sample_cmnts['ncat'] = sample_cmnts['cat'].map(d)
sample_cmnts.head

<bound method NDFrame.head of                          cat  \
0                     sports   
1                     sports   
2                     sports   
3                     sports   
4                     sports   
...                      ...   
9995  science_and_technology   
9996  science_and_technology   
9997  science_and_technology   
9998  science_and_technology   
9999  science_and_technology   

                                                    txt  \
0     Barely better than Gabbert? He was significant...   
1     Fuck the ducks and the Angels! But welcome to ...   
2     Should have drafted more WRs.\n\n- Matt Millen...   
3               [Done](https://i.imgur.com/2YZ90pm.jpg)   
4                                         No!! NOO!!!!!   
...                                                 ...   
9995  Duuude, you're still on that ancient galaxy s3...   
9996  &gt; Moreover, if you buy an unlocked iPhone y...   
9997  Why the hell would I want Bing integrated into..

In [161]:
sample_cmnts

,cat,txt,cleaned,ncat
0,sports,Barely better than Gabbert? He was significant...,barely better than gabbert he was significantl...,1
1,sports,Fuck the ducks and the Angels! But welcome to ...,fuck the ducks and the angels but welcome to a...,1
2,sports,Should have drafted more WRs.\n\n- Matt Millen...,should have drafted more wrs matt millen probably,1
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg),done https i imgur com yz pm jpg,1
4,sports,No!! NOO!!!!!,no noo,1
...,...,...,...,...
9995,science_and_technology,"Duuude, you're still on that ancient galaxy s3...",duuude you re still on that ancient galaxy s ...,2
9996,science_and_technology,"&gt; Moreover, if you buy an unlocked iPhone y...",gt moreover if you buy an unlocked iphone you...,2
9997,science_and_technology,Why the hell would I want Bing integrated into...,why the hell would i want bing integrated into...,2
9998,science_and_technology,"And it overheats like a motherfucker, mostly t...",and it overheats like a motherfucker mostly to...,2


In [162]:
y_output = sample_cmnts['ncat']
print("Unique category:" ,y_output.nunique())

Unique category: 2


In [163]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

number_of_features = 2000

tfidf = TfidfVectorizer(max_features = number_of_features, stop_words=stopwords.words('english'))

X_features = tfidf.fit_transform(sample_cmnts['cleaned']).toarray()
X_features.shape

(10000, 2000)

In [164]:
# Make test and training split (30:70)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(X_features,y_output, random_state=0, test_size = 0.3)

In [165]:
X_train.shape[0]


7000

In [166]:
# Start Neural Network
network = models.Sequential()

In [167]:
# Adding fully connected input layer with a ReLU activation function
network.add(layers.Dense(units=100,
                         activation='relu',
                         input_shape=(number_of_features,)))

In [168]:
# Add fully connected layer with ReLU activation function
network.add(layers.Dense(units=100, activation="relu"))

In [169]:
# Add fully connected layer with a softmax activation function
network.add(layers.Dense(units=3, activation="softmax"))

In [170]:
# Compile neural network
network.compile(loss="categorical_crossentropy", # Cross-entropy
                optimizer="rmsprop", # Root Mean Square Propagation
                metrics=["accuracy"]) # Accuracy performance metric

In [171]:
# Train neural network
history = network.fit(X_train, # Features
                      y_train, # Target
                      epochs=3, # Three epochs
                      verbose=0, # No output
                      batch_size=100, # Number of observations per batch
                      validation_data=(X_test, y_test)) # Test data

ValueError: in user code:

    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
        loss = self.compiled_loss(
    /opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/compile_utils.py:205 __call__
        loss_value = loss_obj(y_t,

In [53]:
features = tfidf_matrix
features.todense()

matrix([[0.07881104, 0.07881104, 0.07881104, 0.31524416, 0.07881104,
         0.07881104, 0.07881104, 0.07881104, 0.07881104, 0.31524416,
         0.07881104, 0.07881104, 0.07881104, 0.07881104, 0.07881104,
         0.07881104, 0.07881104, 0.07881104, 0.07881104, 0.15762208,
         0.07881104, 0.07881104, 0.31524416, 0.15762208, 0.15762208,
         0.15762208, 0.07881104, 0.07881104, 0.07881104, 0.07881104,
         0.07881104, 0.07881104, 0.07881104, 0.15762208, 0.07881104,
         0.15762208, 0.07881104, 0.07881104, 0.07881104, 0.07881104,
         0.07881104, 0.07881104, 0.07881104, 0.07881104, 0.07881104,
         0.07881104, 0.07881104, 0.07881104, 0.07881104, 0.07881104,
         0.07881104, 0.07881104, 0.23643312, 0.07881104, 0.23643312,
         0.07881104, 0.15762208, 0.07881104, 0.23643312, 0.07881104,
         0.15762208, 0.07881104, 0.07881104, 0.15762208, 0.07881104]])

In [54]:
# get the first vector out (for the first document) 
print(features[0]) 

  (0, 0)	0.07881104062391006
  (0, 34)	0.07881104062391006
  (0, 44)	0.07881104062391006
  (0, 56)	0.15762208124782012
  (0, 21)	0.07881104062391006
  (0, 48)	0.07881104062391006
  (0, 4)	0.07881104062391006
  (0, 23)	0.15762208124782012
  (0, 13)	0.07881104062391006
  (0, 37)	0.07881104062391006
  (0, 19)	0.15762208124782012
  (0, 38)	0.07881104062391006
  (0, 59)	0.07881104062391006
  (0, 8)	0.07881104062391006
  (0, 17)	0.07881104062391006
  (0, 28)	0.07881104062391006
  (0, 45)	0.07881104062391006
  (0, 33)	0.15762208124782012
  (0, 7)	0.07881104062391006
  (0, 42)	0.07881104062391006
  (0, 14)	0.07881104062391006
  (0, 26)	0.07881104062391006
  (0, 2)	0.07881104062391006
  (0, 31)	0.07881104062391006
  (0, 27)	0.07881104062391006
  :	:
  (0, 57)	0.07881104062391006
  (0, 3)	0.31524416249564025
  (0, 32)	0.07881104062391006
  (0, 16)	0.07881104062391006
  (0, 39)	0.07881104062391006
  (0, 11)	0.07881104062391006
  (0, 46)	0.07881104062391006
  (0, 41)	0.07881104062391006
  (0, 30)	

In [61]:
# place tf-idf values in a pandas data frame 

# features.T.todense()
# tfidf.get_feature_names()
'''
df = pd.DataFrame(features.T.todense(), index=tfidf.get_feature_names(),columns=[["sam123"]]) 
df1 = df.sort_values(['sam123'], ascending=[True])
df1
'''

'\ndf = pd.DataFrame(features.T.todense(), index=tfidf.get_feature_names(),columns=[["sam123"]]) \ndf1 = df.sort_values([\'sam123\'], ascending=[True])\ndf1\n'